In [ ]:
import os
from os import listdir
from PIL import Image as Image
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
import tensorflow as tf

import pickle
import cv2

In [ ]:
HaarCascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
facenet = load_model('facenet_keras.h5')
model = tf.keras.models.load_model("test.h5")

# load the model from disk
with open('knn_file','rb') as f:
    knn = pickle.load(f)
with open('svm_file','rb') as f:
    svm = pickle.load(f)

In [ ]:
model.summary()

In [ ]:
#load labels
labels = {}
with open("labels.picle", "rb") as f:
    og_labels = pickle.load(f)
    labels = {v:k for k,v in og_labels.items()}

In [ ]:
def get_embedding(facenet, face_pixels):
    face_pixels = face_pixels.astype('float32')
    
    mean,std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels-mean) / std
    
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = facenet.predict(samples)
    return yhat[0]

In [ ]:
def valid_predict(predicao):
    if predicao[0][0] >= 0:
        return True
    if predicao[0][1] >= 0:
        return True
    return False

# KNN

In [ ]:
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Dataset/Teste/Gui/Gui.mp4')
frame_number = 0
try:
    while(True):
        #get frame by frame
        ret, frame = cap.read()

        faces = HaarCascade.detectMultiScale(frame, scaleFactor=1.5, minNeighbors=5)
        for (x, y, w, h) in faces:
            roi = frame[y:y+w, x:x+h,:]
            roi = Image.fromarray(roi)                       
            roi = roi.resize((160,160))
            roi = asarray(roi)
            roi_norm = roi.astype("float32")/255
            emb = get_embedding(facenet, roi_norm)
            tensor = np.expand_dims(emb, axis=0)
            predicao = knn.predict(tensor)
            #confianca = max(round(predicao[0][0],3),round(predicao[0][1],3))*100
            classe = np.argmax(predicao)
            #BOX
            color = (255,0,0) #BGR
            stroke = 2
            width = x + w
            height = y + h
            cv2.rectangle(frame,(x,y), (width, height), color, stroke)
            cv2.putText(frame, labels.get(classe), (x, y+10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            cv2.imwrite('./Dataset/Recognition/' + labels.get(classe)+str(frame_number) + '.jpg',roi)
            frame_number += 1


        #cv2.imshow('.frame',frame)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
except:
    pass

# SVM

In [ ]:
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Dataset/Teste/Gui/Gui.mp4')
frame_number = 0
try:
    while(True):
        #get frame by frame
        ret, frame = cap.read()

        faces = HaarCascade.detectMultiScale(frame, scaleFactor=1.5, minNeighbors=5)
        for (x, y, w, h) in faces:
            roi = frame[y:y+w, x:x+h,:]
            roi = Image.fromarray(roi)                       
            roi = roi.resize((160,160))
            roi = asarray(roi)
            roi_norm = roi.astype("float32")/255
            emb = get_embedding(facenet, roi_norm)
            tensor = np.expand_dims(emb, axis=0)
            #predicao = model.predict(tensor)
            predicao = svm.predict(tensor)
            #confianca = max(round(predicao[0][0],3),round(predicao[0][1],3))*100
            classe = np.argmax(predicao)
            #BOX
            color = (255,0,0) #BGR
            stroke = 2
            width = x + w
            height = y + h
            cv2.rectangle(frame,(x,y), (width, height), color, stroke)
            cv2.putText(frame, labels.get(classe), (x, y+10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            cv2.imwrite('./Dataset/Recognition/' + labels.get(classe)+str(frame_number) + '.jpg',roi)
            frame_number += 1


        cv2.imshow('.frame',frame)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
except:
    pass

# MLP

In [ ]:
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Dataset/Teste/Gui/Gui.mp4')
frame_number = 0
try:
    while(True):
        #get frame by frame
        ret, frame = cap.read()

        faces = HaarCascade.detectMultiScale(frame, scaleFactor=1.5, minNeighbors=5)
        for (x, y, w, h) in faces:
            roi = frame[y:y+w, x:x+h,:]
            roi = Image.fromarray(roi)                       
            roi = roi.resize((160,160))
            roi = asarray(roi)
            roi_norm = roi.astype("float32")/255
            emb = get_embedding(facenet, roi_norm)
            tensor = np.expand_dims(emb, axis=0)
            predicao = model.predict(tensor)
            if valid_predict(predicao):
                confianca = max(round(predicao[0][0],3),round(predicao[0][1],3))*100
                classe = np.argmax(predicao)
                #BOX
                color = (255,0,0) #BGR
                stroke = 2
                width = x + w
                height = y + h
                cv2.rectangle(frame,(x,y), (width, height), color, stroke)
                cv2.putText(frame, labels.get(classe) + " " + str(int(confianca)) + '%', (x, y+10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                cv2.imwrite('./Dataset/Recognition/' + labels.get(classe)+str(int(confianca)) + '%'+str(frame_number) + '.jpg',roi)
                frame_number += 1


        cv2.imshow('.frame',frame)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
except:
    pass